<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_nosql_databases_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial will introduce you to  MongoDB a document database. MongoDB deals in data that is effictively a dictionary in Python or JSON data if it's a file. This document-oriented database is very different from the relational database designs we've seen thus far. We'll learn how to populate the database with data, how to query the database for documents we'd like to find, how to extract data for data engineering purposes.

In [1]:
!pip install faker
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 8.6 MB/s eta 0:00:00


In [2]:
from faker import Faker
from pymongo import MongoClient
import random
import datetime

# install MongoDB

In [3]:
!apt-get install gnupg curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.15).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
!curl -fsSL https://pgp.mongodb.com/server-7.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-7.0.gpg \
   --dearmor

In [5]:
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-7.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse


In [6]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release [2,090 B]
Get:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release.gpg [866 B]
Get:9 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 Packages [22.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Hit:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:13 https://repo.mongodb.org/apt/ubuntu jamm

In [7]:
!apt-get install -y mongodb-org

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org-database mongodb-org-database-tools-extra
  mongodb-org-mongos mongodb-org-server mongodb-org-shell mongodb-org-tools
The following NEW packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server mongodb-org-shell
  mongodb-org-tools
0 upgraded, 9 newly installed, 0 to remove and 24 not upgraded.
Need to get 162 MB of archives.
After this operation, 530 MB of additional disk space will be used.
Get:1 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-database-tools amd64 100.9.4 [51.9 MB]
Get:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-mongosh amd64 2.1.1 [47.9 MB]
Get:3 https://repo.mongodb.org/apt/ub

In [11]:
!mkdir /data
!mkdir /data/db

In [12]:
import subprocess
subprocess.Popen(["mongod"])

<Popen: returncode: None args: ['mongod']>

In [242]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']

['admin', 'cloud_purchase_db', 'config', 'local']

## create the db

name our database

In [281]:
db = client['cloud_purchase_db']

## Set up Collections

drop the collections in case they already exist so we don't duplicate data

In [282]:
db.drop_collection('customers')
db.drop_collection('products')
#db.drop_collection('orders')

{'nIndexesWas': 1, 'ns': 'cloud_purchase_db.products', 'ok': 1.0}

do we have any collections?

## list collections

In [283]:
db.list_collection_names()

[]

In [284]:
# Create Faker instance
fake = Faker()

# Create the data

## create customer data

In [285]:
# Generate customer orders
def gen_orders():

  orders = []
  for i in range(1,random.randint(1, 5)):

      productid = random.randint(1, 10)
      units = random.randint(10, 100)
      if random.random() < 0.1:
          units *= 10
      purchase_date = fake.date_between(start_date='-1y', end_date='today')
      purchase_date = datetime.datetime.combine(purchase_date, datetime.datetime.min.time())

      order = {
          'productid': productid,
          'units': units,
          'purchase_date': purchase_date
      }

      orders.append(order)

  return orders

In [286]:
# Generate customers
customers = []
for i in range(100):
    customer = {
        'customerid': i+1,
        'name': fake.name(),
        'email': fake.email(),
        'phone': [fake.phone_number(),fake.phone_number(),fake.phone_number()],
        'orders' : gen_orders()
    }
    customers.append(customer)

In [287]:
customers[0:3]

[{'customerid': 1,
  'name': 'Tiffany Curtis',
  'email': 'martinezgabriel@example.com',
  'phone': ['+1-307-692-0397', '(283)470-1635x784', '299.225.0915x1132'],
  'orders': [{'productid': 8,
    'units': 55,
    'purchase_date': datetime.datetime(2023, 3, 21, 0, 0)},
   {'productid': 8,
    'units': 37,
    'purchase_date': datetime.datetime(2023, 4, 14, 0, 0)},
   {'productid': 2,
    'units': 26,
    'purchase_date': datetime.datetime(2023, 2, 23, 0, 0)},
   {'productid': 10,
    'units': 46,
    'purchase_date': datetime.datetime(2023, 9, 3, 0, 0)}]},
 {'customerid': 2,
  'name': 'Patrick Cameron',
  'email': 'rebeccasutton@example.net',
  'phone': ['789-276-9551', '558.799.9446x79008', '+1-622-462-1365x24082'],
  'orders': [{'productid': 7,
    'units': 390,
    'purchase_date': datetime.datetime(2023, 10, 15, 0, 0)},
   {'productid': 8,
    'units': 91,
    'purchase_date': datetime.datetime(2023, 2, 12, 0, 0)},
   {'productid': 6,
    'units': 37,
    'purchase_date': datetime.

In [288]:
# Insert customers into MongoDB
db.customers.insert_many(customers)

InsertManyResult([ObjectId('65a1e3db4de10d2f8cecb604'), ObjectId('65a1e3db4de10d2f8cecb605'), ObjectId('65a1e3db4de10d2f8cecb606'), ObjectId('65a1e3db4de10d2f8cecb607'), ObjectId('65a1e3db4de10d2f8cecb608'), ObjectId('65a1e3db4de10d2f8cecb609'), ObjectId('65a1e3db4de10d2f8cecb60a'), ObjectId('65a1e3db4de10d2f8cecb60b'), ObjectId('65a1e3db4de10d2f8cecb60c'), ObjectId('65a1e3db4de10d2f8cecb60d'), ObjectId('65a1e3db4de10d2f8cecb60e'), ObjectId('65a1e3db4de10d2f8cecb60f'), ObjectId('65a1e3db4de10d2f8cecb610'), ObjectId('65a1e3db4de10d2f8cecb611'), ObjectId('65a1e3db4de10d2f8cecb612'), ObjectId('65a1e3db4de10d2f8cecb613'), ObjectId('65a1e3db4de10d2f8cecb614'), ObjectId('65a1e3db4de10d2f8cecb615'), ObjectId('65a1e3db4de10d2f8cecb616'), ObjectId('65a1e3db4de10d2f8cecb617'), ObjectId('65a1e3db4de10d2f8cecb618'), ObjectId('65a1e3db4de10d2f8cecb619'), ObjectId('65a1e3db4de10d2f8cecb61a'), ObjectId('65a1e3db4de10d2f8cecb61b'), ObjectId('65a1e3db4de10d2f8cecb61c'), ObjectId('65a1e3db4de10d2f8cecb6

## create products data

In [289]:
# Generate products
products = []
for i in range(10):
    product = {
        'productid': i+1,
        'category': random.choice(['Electronics', 'Clothing', 'Books', 'Home']),
        'price': random.randint(1, 100)
    }
    products.append(product)

In [290]:
# Insert products into MongoDB
db.products.insert_many(products)

InsertManyResult([ObjectId('65a1e3db4de10d2f8cecb668'), ObjectId('65a1e3db4de10d2f8cecb669'), ObjectId('65a1e3db4de10d2f8cecb66a'), ObjectId('65a1e3db4de10d2f8cecb66b'), ObjectId('65a1e3db4de10d2f8cecb66c'), ObjectId('65a1e3db4de10d2f8cecb66d'), ObjectId('65a1e3db4de10d2f8cecb66e'), ObjectId('65a1e3db4de10d2f8cecb66f'), ObjectId('65a1e3db4de10d2f8cecb670'), ObjectId('65a1e3db4de10d2f8cecb671')], acknowledged=True)

In [291]:
db.list_collection_names() # list collections

['customers', 'products']

loop through collections and count the number of documents

In [292]:
for collection_name in db.list_collection_names():
  collection = db.get_collection(collection_name)
  print(f'{collection_name}:{collection.count_documents({})}')

customers:100
products:10


# Querying



## Customers

Find all customers.

In [293]:
for document in db.customers.find({}):
  print(document)

{'_id': ObjectId('65a1e3db4de10d2f8cecb604'), 'customerid': 1, 'name': 'Tiffany Curtis', 'email': 'martinezgabriel@example.com', 'phone': ['+1-307-692-0397', '(283)470-1635x784', '299.225.0915x1132'], 'orders': [{'productid': 8, 'units': 55, 'purchase_date': datetime.datetime(2023, 3, 21, 0, 0)}, {'productid': 8, 'units': 37, 'purchase_date': datetime.datetime(2023, 4, 14, 0, 0)}, {'productid': 2, 'units': 26, 'purchase_date': datetime.datetime(2023, 2, 23, 0, 0)}, {'productid': 10, 'units': 46, 'purchase_date': datetime.datetime(2023, 9, 3, 0, 0)}]}
{'_id': ObjectId('65a1e3db4de10d2f8cecb605'), 'customerid': 2, 'name': 'Patrick Cameron', 'email': 'rebeccasutton@example.net', 'phone': ['789-276-9551', '558.799.9446x79008', '+1-622-462-1365x24082'], 'orders': [{'productid': 7, 'units': 390, 'purchase_date': datetime.datetime(2023, 10, 15, 0, 0)}, {'productid': 8, 'units': 91, 'purchase_date': datetime.datetime(2023, 2, 12, 0, 0)}, {'productid': 6, 'units': 37, 'purchase_date': datetime.

find customerid 76

In [294]:
import pprint

In [295]:
for document in db.customers.find({'customerid':76}):
  print(pprint.pprint(document))

{'_id': ObjectId('65a1e3db4de10d2f8cecb64f'),
 'customerid': 76,
 'email': 'bswanson@example.com',
 'name': 'Sabrina Cook',
 'orders': [{'productid': 7,
             'purchase_date': datetime.datetime(2023, 4, 7, 0, 0),
             'units': 95},
            {'productid': 4,
             'purchase_date': datetime.datetime(2023, 5, 16, 0, 0),
             'units': 540}],
 'phone': ['(679)597-9892x964', '317-647-5953', '(667)587-8665x30494']}
None


## Products

find all products

In [296]:
for document in db.products.find({}):
  print(document)

{'_id': ObjectId('65a1e3db4de10d2f8cecb668'), 'productid': 1, 'category': 'Home', 'price': 58}
{'_id': ObjectId('65a1e3db4de10d2f8cecb669'), 'productid': 2, 'category': 'Electronics', 'price': 2}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66a'), 'productid': 3, 'category': 'Home', 'price': 80}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66b'), 'productid': 4, 'category': 'Electronics', 'price': 32}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66c'), 'productid': 5, 'category': 'Clothing', 'price': 75}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66d'), 'productid': 6, 'category': 'Electronics', 'price': 32}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66e'), 'productid': 7, 'category': 'Books', 'price': 98}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66f'), 'productid': 8, 'category': 'Books', 'price': 63}
{'_id': ObjectId('65a1e3db4de10d2f8cecb670'), 'productid': 9, 'category': 'Home', 'price': 30}
{'_id': ObjectId('65a1e3db4de10d2f8cecb671'), 'productid': 10, 'category': 'Home', 'price': 76}


find all products with prices less than 40

In [297]:
for document in db.products.find({'price': {'$lt':40}}):
  print(document)

{'_id': ObjectId('65a1e3db4de10d2f8cecb669'), 'productid': 2, 'category': 'Electronics', 'price': 2}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66b'), 'productid': 4, 'category': 'Electronics', 'price': 32}
{'_id': ObjectId('65a1e3db4de10d2f8cecb66d'), 'productid': 6, 'category': 'Electronics', 'price': 32}
{'_id': ObjectId('65a1e3db4de10d2f8cecb670'), 'productid': 9, 'category': 'Home', 'price': 30}


find all products with category clothing

In [298]:
for document in db.products.find({'category': 'Clothing'}):
  print(document)

{'_id': ObjectId('65a1e3db4de10d2f8cecb66c'), 'productid': 5, 'category': 'Clothing', 'price': 75}


find all products with price less than 40 AND category Clothing

In [299]:
for document in db.products.find({'price': {'$lt':40},'category': 'Clothing'}):
  print(document)

## Orders

In [300]:
# Calculate the total number of orders
pipeline = [
    {
        '$unwind': '$orders'
    },
    {
        '$group': {
            '_id': None,
            'total_orders': {'$sum': 1}
        }
    },
    {
        '$project': {
            '_id': 0,
            'total_orders': 1
        }
    }
]

result = db.customers.aggregate(pipeline)

# Extract the total number of orders
total_orders = next(result)['total_orders']

# Print the total number of orders
print(f"Total Orders: {total_orders}")

Total Orders: 190


In [301]:
filter_criteria = {
    'orders.units': {'$lt': 45000}
}

# Calculate the total number of orders with units less than 12000
pipeline = [
    {
      '$unwind': '$orders'
    },
    {
      '$match': filter_criteria
    },
    {
        '$group': {
            '_id': None,
            'total_orders': {'$sum': 1}
        }
    },
    {
        '$project': {
            '_id': 0,
            'total_orders': 1
        }
    }
]

result = db.customers.aggregate(pipeline)

# Extract the total number of orders
total_orders = next(result)['total_orders']

# Print the total number of orders
print(f"Total Orders: {total_orders}")

Total Orders: 190


# Extraction


## Dump mongodb data to json file.

create a JSON file with the filtered data.

In [302]:
from bson.json_util import dumps
import json

open a file. create a cursor that gets passed to dumps which takes binary json data and converts it serialized json, deseralizes it to python object and then serializes it back to a file.  

## dump orders

## dump customers

(and orders as they are nested)

In [303]:
with open('customers.json', 'w') as file:
  cursor = db.customers.find({})
  file.write(dumps(cursor))

## dump products

In [304]:
with open('products.json', 'w') as file:
  cursor = db.products.find({})
  file.write(dumps(cursor))

In [305]:
!ls -lh *.json

-rw-r--r-- 1 root root 36K Jan 13 01:14 customers.json
-rw-r--r-- 1 root root 987 Jan 13 01:14 products.json


# Reading JSON file data into Python

## JSON into Pandas DataFrame

In [306]:
import pandas as pd

In [307]:
customers_df = pd.read_json('customers.json')
customers_df.head(2)

,_id,customerid,name,email,phone,orders
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...","[{'productid': 8, 'units': 55, 'purchase_date'..."
1,{'$oid': '65a1e3db4de10d2f8cecb605'},2,Patrick Cameron,rebeccasutton@example.net,"[789-276-9551, 558.799.9446x79008, +1-622-462-...","[{'productid': 7, 'units': 390, 'purchase_date..."


In [308]:
products_df = pd.read_json('products.json')
products_df.head(2)

,_id,productid,category,price
0,{'$oid': '65a1e3db4de10d2f8cecb668'},1,Home,58
1,{'$oid': '65a1e3db4de10d2f8cecb669'},2,Electronics,2


### Merge DataFrames

In [309]:
customers_df.head(4)

,_id,customerid,name,email,phone,orders
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...","[{'productid': 8, 'units': 55, 'purchase_date'..."
1,{'$oid': '65a1e3db4de10d2f8cecb605'},2,Patrick Cameron,rebeccasutton@example.net,"[789-276-9551, 558.799.9446x79008, +1-622-462-...","[{'productid': 7, 'units': 390, 'purchase_date..."
2,{'$oid': '65a1e3db4de10d2f8cecb606'},3,Wesley Schroeder,alexisgreen@example.org,"[239-429-9960x246, 793.805.2356, 001-794-937-5...","[{'productid': 1, 'units': 21, 'purchase_date'..."
3,{'$oid': '65a1e3db4de10d2f8cecb607'},4,Aaron Mendez,masonjamie@example.com,"[535.526.2613x3518, (501)717-5312, 332.558.3854]","[{'productid': 1, 'units': 14, 'purchase_date'..."


In [310]:
orders_df = customers_df.explode('orders')

In [311]:
df_orders_expanded = pd.concat([orders_df.drop(['orders'], axis=1), orders_df['orders'].apply(pd.Series)], axis=1)
df_orders_expanded.head(3)


,_id,customerid,name,email,phone,productid,units,purchase_date,0
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",8.0,55.0,{'$date': '2023-03-21T00:00:00Z'},NaN
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",8.0,37.0,{'$date': '2023-04-14T00:00:00Z'},NaN
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",2.0,26.0,{'$date': '2023-02-23T00:00:00Z'},NaN


In [312]:
customer_product_orders = df_orders_expanded.merge(products_df, on='productid', how='left')
customer_product_orders.head(3)

,_id_x,customerid,name,email,phone,productid,units,purchase_date,0,_id_y,category,price
0,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",8.0,55.0,{'$date': '2023-03-21T00:00:00Z'},NaN,{'$oid': '65a1e3db4de10d2f8cecb66f'},Books,63.0
1,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",8.0,37.0,{'$date': '2023-04-14T00:00:00Z'},NaN,{'$oid': '65a1e3db4de10d2f8cecb66f'},Books,63.0
2,{'$oid': '65a1e3db4de10d2f8cecb604'},1,Tiffany Curtis,martinezgabriel@example.com,"[+1-307-692-0397, (283)470-1635x784, 299.225.0...",2.0,26.0,{'$date': '2023-02-23T00:00:00Z'},NaN,{'$oid': '65a1e3db4de10d2f8cecb669'},Electronics,2.0


In [313]:
customer_product_orders['total_sales'] = customer_product_orders['price'] * customer_product_orders['units']

In [314]:
customer_product_orders.groupby(by='category').agg({'total_sales': 'sum'}).sort_values(by='total_sales', ascending=False)

,total_sales
category,
Home,556980.0
Books,481495.0
Electronics,198420.0
Clothing,148350.0
